In [1]:
%%javascript
Jupyter.utils.load_extensions('tdb_ext/main')

<IPython.core.display.Javascript object>

In [2]:
#this sets the backend to jupyter/ipython that (i think) displays
#     images directly. anyway, it prevents the matplotlib framework
#     python error that is my least favorite thing eeeevvvveeeer.
%matplotlib notebook

import sys
sys.path.append("..")

import gmix_model as gmix
import numpy as np
import tdb as tdb
import tensorflow as tf
import matplotlib.pyplot as plt

import os
os.chdir('/Users/azane/GitRepo/spider')

In [3]:
#read in training and test data
s_x, s_t = gmix.get_xt_from_npz('data/toy_train.npz')
t_x, t_t = gmix.get_xt_from_npz('data/toy_train.npz')

In [4]:
#create gmm with data
gmm = gmix.GaussianMixtureModel(s_x, s_t, t_x, t_t,
                               numGaussianComponents=3, hiddenLayerSize=5,
                               learningRate=0.01)

In [5]:
%%capture
d = gmm.train(iterations=50, testBatchSize=300,
          trainBatchSize=50, reportEvery=5)

In [6]:
%%capture
def x1_yMany(x, y, xLabel='', yLabel='', title=''):
    
    #x.shape == [s,x]
    #y.shape == [s,t/y]
    
    #copy x and y before manipulating them
    x = np.copy(x)
    y = np.copy(y)
    
    
    x_hold = x[:,0]  # select the 1st input dimension
    
    #iterate the output dimensions, plotting each line with the original x values.
    for outDim in range(y.shape[1]):
        y_ = y[:,outDim]
        x_, y_ = sort_by_x(x_hold, y_)
        plt.plot(x_, y_)
    
    plt.xlabel(xLabel)
    plt.ylabel(yLabel)
    plt.title(title)

def sort_by_x(x, y):
    
    #copy x and y before manipulating them
    x = np.copy(x)
    y = np.copy(y)
    
    #x and y should be 1d at this point .shape == [s,]
    xy = np.column_stack((x,y))  # stack horizontally
    xy = xy[xy[:,0].argsort()]  # sort by the x column
    return xy[:,0], xy[:,1]  # re-separate x and y

x1_yMany(d['netIn'], d['lay1'])

In [7]:
s = 200  # sample index for testing

In [8]:
print 'netIn:'
print d['netIn'][s]

small_change = d['netIn'][s] + 0.05
print 'small_change:'
print small_change


netIn:
[-0.43225092]
small_change:
[-0.38225091]


In [9]:
print 'w1:'
print d['w1']
print 'b1:'
print d['b1']
print 'lay1:'
print d['lay1'][s]


np_calc1 =  np.tanh( np.dot(d['netIn'][s], d['w1']) + d['b1'])
np_dif1 =  np_calc1 - d['lay1'][s]

print 'numpy calc:'
print np_calc1
print 'numpy calc dif:'
print np_dif1

print '------------------------------'
np_calc_sc1 = np.tanh( np.dot(small_change, d['w1']) + d['b1'])
np_dif_sc1 = np_calc1 - np_calc_sc1

print 'numpy calc:'
print np_calc1
print 'numpy calc of small change:'
print np_calc_sc1
print 'numpy calc dif with small change:'
print np_dif_sc1
print '------------------------------'


w1:
[[ 0.40667036  0.28458923 -0.89801604 -0.79920626  0.81469202]]
b1:
[-0.11351877 -0.18813045 -0.00328432  0.13726676 -0.04598494]
lay1:
[-0.28149259 -0.30147791  0.36694139  0.44842285 -0.37835324]
numpy calc:
[-0.28149262 -0.30147785  0.36694136  0.44842285 -0.37835321]
numpy calc dif:
[ -2.98023224e-08   5.96046448e-08  -2.98023224e-08   0.00000000e+00
   2.98023224e-08]
------------------------------
numpy calc:
[-0.28149262 -0.30147785  0.36694136  0.44842285 -0.37835321]
numpy calc of small change:
[-0.26266512 -0.28848684  0.32746232  0.41593295 -0.34292334]
numpy calc dif with small change:
[-0.0188275  -0.01299101  0.03947905  0.0324899  -0.03542987]
------------------------------


In [10]:
print 'w2:'
print d['w2']
print 'b2:'
print d['b2']
print 'lay2:'
print d['lay2'][s]


np_calc2 =  np.tanh( np.dot(np_calc1, d['w2']) + d['b2'])
np_dif2 =  np_calc2 - d['lay2'][s]

print 'numpy calc:'
print np_calc2
print 'numpy calc dif:'
print np_dif2


w2:
[[ 0.50577658  0.01776878 -0.67997313  0.24501085  0.27735576]
 [-0.63150376  0.12643455 -0.00975736  0.32009915  0.05354896]
 [-0.33654007 -0.12411223 -0.27566034  0.16767456 -0.63514268]
 [-0.19004287 -0.35363439 -0.63988948 -0.54241627 -0.76692641]
 [-0.30518848  0.75420862 -0.30047292  0.07782117  0.61024308]]
b2:
[ 0.35622072 -0.03145356  0.30984119 -0.56983423  0.08022787]
lay2:
[ 0.30133909 -0.51097572  0.22582227 -0.73817384 -0.67607248]
numpy calc:
[ 0.30133906 -0.51097578  0.22582234 -0.73817384 -0.67607254]
numpy calc dif:
[ -2.98023224e-08  -5.96046448e-08   7.45058060e-08   0.00000000e+00
  -5.96046448e-08]


In [11]:
print 'w3:'
print d['w3']
print 'b3:'
print d['b3']
print 'netOut:'
print d['netOut'][s]


np_calc3 =  np.tanh( np.dot(np_calc2, d['w3']) + d['b3'])
np_dif3 =  np_calc3 - d['netOut'][s]

print 'numpy calc:'
print np_calc3
print 'numpy calc dif:'
print np_dif3

w3:
[[ 0.24601658 -0.0787788  -0.32515535 -0.31669205  0.27309188 -0.61335355
  -0.22778022]
 [ 0.84349132 -0.45599306  0.11260852  0.435835    0.20286159 -0.52295196
   0.08636043]
 [-0.23068745 -0.34207383  0.00918611 -0.26131561  0.48594093  0.01415674
  -0.27295873]
 [-0.5729562  -0.51890928 -0.90473533  0.57074738 -0.79113871 -0.0204827
   0.74874318]
 [ 0.61222327 -0.0288803   0.20458098 -0.12409408  0.54444742  0.36445695
   0.76775938]]
b3:
[ 0.06115909  0.09266346  0.32325372 -1.26113832  0.2243796  -0.15214485
 -0.25548941]
netOut:
[-0.3263793   0.55615449  0.60395265 -0.96226978  0.48435897 -0.28933382
 -0.90544772]
numpy calc:
[-0.32637936  0.55615455  0.60395265 -0.96226966  0.484359   -0.28933385
 -0.90544784]
numpy calc dif:
[ -5.96046448e-08   5.96046448e-08   0.00000000e+00   1.19209290e-07
   2.98023224e-08  -2.98023224e-08  -1.19209290e-07]
